In [8]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import random
import pandas as pd
import seaborn as sns
import json
from sklearn.model_selection import KFold
import certifi

#***************** Setup Environment *************************
os.environ['SSL_CERT_FILE'] = certifi.where()
os.environ['REQUESTS_CA_BUNDLE'] = certifi.where()

# Check if MPS is available for Mac
print("PyTorch version:", torch.__version__)
print("MPS available:", torch.backends.mps.is_available())

# Set device - use MPS if available, otherwise fall back to CPU or CUDA
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS device")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using {device} device")

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Paths
DATASET_PATH = "../archive/images/images"
RESULTS_DIR = "resnet50_results"
os.makedirs(RESULTS_DIR, exist_ok=True)

#***************** Hyperparameters *************************
# Define hyperparameter space
hyperparams = {
    'learning_rate': [1e-3, 5e-4, 1e-4],
    'batch_size': [16, 32, 64],
    'optimizer': ['Adam', 'SGD'],
    'dropout_rate': [0.2, 0.5],
    'unfreeze_layers': ['none', 'last_block', 'all']
}

#***************** Data Transformations *************************
def get_transforms():
    train_transform = transforms.Compose([
        transforms.Resize((240, 240)),
        transforms.RandomCrop((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    return train_transform, val_transform

#***************** Data Loading *************************
def load_data(batch_size=32):
    train_transform, val_transform = get_transforms()
    
    full_dataset = datasets.ImageFolder(DATASET_PATH, transform=train_transform)
    class_names = full_dataset.classes
    num_classes = len(class_names)
    
    # Get dataset size
    dataset_size = len(full_dataset)
    
    # Create indices for the splits
    indices = list(range(dataset_size))
    np.random.shuffle(indices)
    
    # Calculate split sizes
    test_split = int(0.1 * dataset_size)  # 10% for testing
    val_split = int(0.2 * dataset_size)   # 20% for validation
    
    # Create the indices for each split
    test_indices = indices[:test_split]
    val_indices = indices[test_split:(test_split + val_split)]
    train_indices = indices[(test_split + val_split):]
    
    # Create samplers
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)
    test_sampler = SubsetRandomSampler(test_indices)
    
    # Create datasets with appropriate transforms
    val_dataset = datasets.ImageFolder(DATASET_PATH, transform=val_transform)
    test_dataset = datasets.ImageFolder(DATASET_PATH, transform=val_transform)
    
    # Create data loaders
    train_loader = DataLoader(full_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, sampler=val_sampler, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, sampler=test_sampler, num_workers=0)
    
    print(f"Dataset loaded: {len(train_indices)} training, {len(val_indices)} validation, {len(test_indices)} test images")
    print(f"Classes: {class_names}")
    
    return train_loader, val_loader, test_loader, class_names, num_classes

#***************** Model Creation *************************
def create_model(num_classes, dropout_rate=0.5):
    # Load pretrained ResNet-50 model
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
    
    # First freeze all parameters
    for param in model.parameters():
        param.requires_grad = False
    
    # Replace the final fully connected layer for our classification task
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(p=dropout_rate),
        nn.Linear(num_features, num_classes)
    )
    
    return model

#***************** Layer Unfreezing *************************
def set_trainable_layers(model, unfreeze_layers):
    # First freeze all parameters
    for param in model.parameters():
        param.requires_grad = False
    
    # Always unfreeze the final classifier
    for param in model.fc.parameters():
        param.requires_grad = True
    
    if unfreeze_layers == 'last_block':
        # Unfreeze the last ResNet layer
        for param in model.layer4.parameters():
            param.requires_grad = True
    elif unfreeze_layers == 'all':
        # Unfreeze all layers
        for param in model.parameters():
            param.requires_grad = True
    
    return model

#***************** Early Stopping *************************
class EarlyStopping:
    def __init__(self, patience=10, verbose=True, path='best_model.pth'):
        self.patience = patience
        self.verbose = verbose
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        
    def __call__(self, val_loss, model):
        if self.best_score is None or -val_loss > self.best_score:
            self.best_score = -val_loss
            self.save_checkpoint(val_loss, model)
            self.counter = 0
        else:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
    
    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

#***************** Training Function *************************
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=None, num_epochs=30, path='model.pth'):
    early_stopping = EarlyStopping(patience=10, path=path)
    
    # History tracking
    history = {
        'train_loss': [], 'val_loss': [], 
        'train_acc': [], 'val_acc': [], 
        'f1': []
    }
    
    model = model.to(device)
    start_time = time.time()
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        
        # Training phase
        model.train()
        running_loss = 0.0
        all_train_preds = []
        all_train_labels = []
        total_train_processed = 0
        
        for inputs, labels in tqdm(train_loader, desc=f"Training"):
            inputs, labels = inputs.to(device), labels.to(device)
            batch_size = inputs.size(0)
            total_train_processed += batch_size
            
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * batch_size
            all_train_preds.extend(preds.cpu().numpy())
            all_train_labels.extend(labels.cpu().numpy())
        
        # Calculate training metrics correctly
        correct_train = sum(np.array(all_train_preds) == np.array(all_train_labels))
        epoch_loss = running_loss / total_train_processed
        epoch_acc = correct_train / total_train_processed
        
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc)
        
        
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        
        # Validation phase
        model.eval()
        running_loss = 0.0
        all_preds = []
        all_labels = []
        total_processed = 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Validation"):
                inputs, labels = inputs.to(device), labels.to(device)
                batch_size = inputs.size(0)  # Get actual batch size
                total_processed += batch_size
                
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                running_loss += loss.item() * batch_size
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        # Calculate metrics using the arrays of all predictions and labels
        correct_preds = sum(np.array(all_preds) == np.array(all_labels))
        epoch_loss = running_loss / total_processed
        epoch_acc = correct_preds / total_processed
        f1 = f1_score(all_labels, all_preds, average='weighted')
        
        history['val_loss'].append(epoch_loss)
        history['val_acc'].append(epoch_acc)
        history['f1'].append(f1)
        
        print(f'Val Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} F1: {f1:.4f}')
        
        # Update scheduler if provided
        if scheduler is not None:
            scheduler.step(epoch_loss)
        
        # Early stopping check
        early_stopping(epoch_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
    
    # Calculate training time
    time_elapsed = time.time() - start_time
    print(f'Training completed in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    
    # Load best model
    model.load_state_dict(torch.load(early_stopping.path))
    
    return model, history

#***************** Evaluation Function *************************
def evaluate_model(model, data_loader, class_names, save_path=None):
    model.eval()
    all_preds = []
    all_labels = []
    total_samples = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            batch_size = inputs.size(0)
            total_samples += batch_size
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Convert lists to numpy arrays
    all_preds_np = np.array(all_preds)
    all_labels_np = np.array(all_labels)
    
    # Calculate metrics
    correct_predictions = np.sum(all_preds_np == all_labels_np)
    accuracy = correct_predictions / len(all_preds_np)
    
    cm = confusion_matrix(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    report_str = classification_report(all_labels, all_preds, target_names=class_names)
    
    # Print report
    print(f'Evaluation Accuracy: {accuracy:.4f}')
    print('\nClassification Report:')
    print(report_str)
    
    if save_path:
        # Save detailed report
        with open(f"{save_path}_report.txt", 'w') as f:
            f.write(f'Evaluation Accuracy: {accuracy:.4f}\n\n')
            f.write(report_str)
            
        # Save report as JSON for further analysis
        with open(f"{save_path}_report.json", 'w') as f:
            json.dump(report, f, indent=4)
        
        # Plot confusion matrix
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix')
        plt.tight_layout()
        plt.savefig(f"{save_path}_confusion_matrix.png")
        plt.close()
        
        print(f'Evaluation results saved to {save_path}')
    
    return accuracy, report
    
    # Print report
    print(f'Evaluation Accuracy: {accuracy:.4f}')
    print('\nClassification Report:')
    print(report_str)
    
    if save_path:
        # Save detailed report
        with open(f"{save_path}_report.txt", 'w') as f:
            f.write(f'Evaluation Accuracy: {accuracy:.4f}\n\n')
            f.write(report_str)
            
        # Save report as JSON for further analysis
        with open(f"{save_path}_report.json", 'w') as f:
            json.dump(report, f, indent=4)
        
        # Plot confusion matrix
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix')
        plt.tight_layout()
        plt.savefig(f"{save_path}_confusion_matrix.png")
        plt.close()
        
        print(f'Evaluation results saved to {save_path}')
    
    return accuracy, report

#***************** Plotting Functions *************************
def plot_history(history, save_path=None):
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 3, 2)
    plt.plot(history['train_acc'], label='Train Acc')
    plt.plot(history['val_acc'], label='Val Acc')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 3, 3)
    plt.plot(history['f1'], label='F1 Score')
    plt.title('F1 Score')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(f"{save_path}_history.png")
        print(f'Training history plot saved to {save_path}_history.png')
    
    plt.close()

def plot_hp_search_results(results_df, save_dir):
    # Create plots to visualize hyperparamater search results
    
    # Plot accuracy by trial
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['trial'], results_df['accuracy'], 'o-', label='Accuracy')
    plt.plot(results_df['trial'], results_df['f1'], 's-', label='F1 Score')
    plt.axhline(y=results_df['accuracy'].max(), color='r', linestyle='--', 
               label=f'Best Accuracy: {results_df["accuracy"].max():.4f}')
    plt.xlabel('Trial')
    plt.ylabel('Score')
    plt.title('Performance by Trial')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(save_dir, 'performance_by_trial.png'))
    plt.close()
    
    # Plot parameters influence
    plt.figure(figsize=(15, 10))
    
    plt.subplot(2, 2, 1)
    sns.boxplot(x='learning_rate', y='accuracy', data=results_df)
    plt.title('Accuracy by Learning Rate')
    plt.grid(True)
    
    plt.subplot(2, 2, 2)
    sns.boxplot(x='batch_size', y='accuracy', data=results_df)
    plt.title('Accuracy by Batch Size')
    plt.grid(True)
    
    plt.subplot(2, 2, 3)
    sns.boxplot(x='optimizer', y='accuracy', data=results_df)
    plt.title('Accuracy by Optimizer')
    plt.grid(True)
    
    plt.subplot(2, 2, 4)
    sns.boxplot(x='unfreeze_layers', y='accuracy', data=results_df)
    plt.title('Accuracy by Unfreezing Strategy')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'parameter_influence.png'))
    plt.close()
    
    # Save summary stats
    summary_stats = results_df.groupby(['learning_rate', 'optimizer', 'unfreeze_layers'])[['accuracy', 'f1']].agg(['mean', 'std', 'max'])
    summary_stats.to_csv(os.path.join(save_dir, 'parameter_summary_stats.csv'))
    
    print(f"Hyperparameter search summary saved to {save_dir}")

#***************** Hyperparameter Search *************************
def run_hyperparameter_search(num_trials=15):
    # Load data to get class names and num_classes
    train_loader, val_loader, _, class_names, num_classes = load_data(batch_size=32)
    
    # Generate random hyperparameter combinations
    trials = []
    for i in range(num_trials):
        params = {
            'learning_rate': random.choice(hyperparams['learning_rate']),
            'batch_size': random.choice(hyperparams['batch_size']),
            'optimizer': random.choice(hyperparams['optimizer']),
            'dropout_rate': random.choice(hyperparams['dropout_rate']),
            'unfreeze_layers': random.choice(hyperparams['unfreeze_layers'])
        }
        trials.append(params)
    
    # Keep track of results
    results = []
    
    # Run each trial
    for i, params in enumerate(trials):
        trial_id = i + 1
        print(f"\n{'='*50}")
        print(f"Trial {trial_id}/{num_trials}")
        print(f"Parameters: {params}")
        print(f"{'='*50}")
        
        # Set a new seed for this trial
        trial_seed = 42 + trial_id
        torch.manual_seed(trial_seed)
        np.random.seed(trial_seed)
        random.seed(trial_seed)
        
        # Load data with current batch size
        train_loader, val_loader, _, _, _ = load_data(batch_size=params['batch_size'])
        
        # Create model
        model = create_model(num_classes, dropout_rate=params['dropout_rate'])
        model = set_trainable_layers(model, params['unfreeze_layers'])
        
        # Setup optimizer
        if params['optimizer'] == 'Adam':
            optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                                  lr=params['learning_rate'])
        else:
            optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=params['learning_rate'], momentum=0.9)
        
        # Setup loss and scheduler
        criterion = nn.CrossEntropyLoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                        factor=0.5, patience=5, verbose=True)
        
        # Create directory for this trial
        trial_dir = os.path.join(RESULTS_DIR, f"trial_{trial_id}")
        os.makedirs(trial_dir, exist_ok=True)
        
        # Train model
        model_path = os.path.join(trial_dir, f'model_trial_{trial_id}.pth')
        model, history = train_model(model, train_loader, val_loader, criterion, optimizer, 
                                     scheduler=scheduler, num_epochs=30, path=model_path)
        
        # Plot and save training history
        plot_history(history, save_path=os.path.join(trial_dir, f"trial_{trial_id}"))
        
        # Evaluate model
        accuracy, report = evaluate_model(model, val_loader, class_names, 
                                         save_path=os.path.join(trial_dir, f"trial_{trial_id}"))
        
        # Save hyperparameters
        with open(os.path.join(trial_dir, 'hyperparameters.json'), 'w') as f:
            json.dump(params, f, indent=4)
        
        # Save results
        trial_result = {
            'trial': trial_id,
            'accuracy': accuracy,
            'f1': report['weighted avg']['f1-score'],
            'learning_rate': params['learning_rate'],
            'batch_size': params['batch_size'],
            'optimizer': params['optimizer'],
            'dropout_rate': params['dropout_rate'],
            'unfreeze_layers': params['unfreeze_layers'],
            'model_path': model_path
        }
        results.append(trial_result)
        
        # Save intermediate results
        results_df = pd.DataFrame(results)
        results_df.to_csv(os.path.join(RESULTS_DIR, 'hp_search_results.csv'), index=False)
    
    # Find best hyperparameters
    results_df = pd.DataFrame(results)
    best_idx = results_df['accuracy'].idxmax()
    best_params = results_df.iloc[best_idx].to_dict()
    
    # Save best params
    with open(os.path.join(RESULTS_DIR, 'best_params.json'), 'w') as f:
        json.dump(best_params, f, indent=4)
    
    # Create summary plots
    summary_dir = os.path.join(RESULTS_DIR, "hp_search_summary")
    os.makedirs(summary_dir, exist_ok=True)
    plot_hp_search_results(results_df, summary_dir)
    
    print("\n" + "="*50)
    print("Hyperparameter Search Complete")
    print(f"Best accuracy: {best_params['accuracy']:.4f}")
    print(f"Best F1 score: {best_params['f1']:.4f}")
    print(f"Best parameters: {best_params}")
    
    return best_params

#***************** Final Model Training *************************
def train_final_model(best_params):
    print("\n" + "="*50)
    print("Training Final Model with Best Hyperparameters")
    print(f"Parameters: {best_params}")
    
    # Create directory for final model
    final_dir = os.path.join(RESULTS_DIR, "final_model")
    os.makedirs(final_dir, exist_ok=True)
    
    # Load data
    train_loader, val_loader, test_loader, class_names, num_classes = load_data(
        batch_size=int(best_params['batch_size']))
    
    # Create model
    model = create_model(num_classes, dropout_rate=best_params['dropout_rate'])
    model = set_trainable_layers(model, best_params['unfreeze_layers'])
    
    # Setup optimizer
    if best_params['optimizer'] == 'Adam':
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                              lr=best_params['learning_rate'])
    else:
        optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                             lr=best_params['learning_rate'], momentum=0.9)
    
    # Setup loss and scheduler
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                    factor=0.5, patience=5, verbose=True)
    
    # Train final model with more epochs
    final_model_path = os.path.join(final_dir, 'final_model.pth')
    model, history = train_model(model, train_loader, val_loader, criterion, optimizer, 
                                 scheduler=scheduler, num_epochs=50, path=final_model_path)
    
    # Plot training history
    plot_history(history, save_path=os.path.join(final_dir, "final_model"))
    
    # Evaluate on validation set
    val_accuracy, val_report = evaluate_model(model, val_loader, class_names, 
                                     save_path=os.path.join(final_dir, "final_model_validation"))
    
    # Evaluate on test set
    test_accuracy, test_report = evaluate_model(model, test_loader, class_names, 
                                      save_path=os.path.join(final_dir, "final_model_test"))
    
    # Save model architecture details
    model_summary = {
        'model_type': 'ResNet-50',
        'num_classes': num_classes,
        'class_names': class_names,
        'parameters': sum(p.numel() for p in model.parameters()),
        'trainable_parameters': sum(p.numel() for p in model.parameters() if p.requires_grad),
        'hyperparameters': best_params,
        'validation_accuracy': val_accuracy,
        'validation_f1': val_report['weighted avg']['f1-score'],
        'test_accuracy': test_accuracy,
        'test_f1': test_report['weighted avg']['f1-score'],
        'per_class_f1': {cls: test_report[cls]['f1-score'] for cls in class_names}
    }
    
    with open(os.path.join(final_dir, 'model_details.json'), 'w') as f:
        json.dump(model_summary, f, indent=4)
    
    print(f"\nFinal model saved to {final_model_path}")
    print(f"Final validation accuracy: {val_accuracy:.4f}")
    print(f"Final validation F1 score: {val_report['weighted avg']['f1-score']:.4f}")
    print(f"Final test accuracy: {test_accuracy:.4f}")
    print(f"Final test F1 score: {test_report['weighted avg']['f1-score']:.4f}")
    
    return model, test_accuracy, test_report

#***************** Cross-Validation *************************
def run_cross_validation(best_params, k_folds=5):
    print("\n" + "="*50)
    print(f"Running {k_folds}-fold Cross-Validation with Best Parameters")
    
    # Create directory for CV results
    cv_dir = os.path.join(RESULTS_DIR, "cross_validation")
    os.makedirs(cv_dir, exist_ok=True)
    
    # Load data for entire dataset
    train_transform, val_transform = get_transforms()
    full_dataset = datasets.ImageFolder(DATASET_PATH, transform=train_transform)
    val_dataset = datasets.ImageFolder(DATASET_PATH, transform=val_transform)
    class_names = full_dataset.classes
    num_classes = len(class_names)
    
    # Create KFold
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    
    # Results storage
    fold_results = []
    all_metrics = []
    
    # Run k-fold cross-validation
    for fold, (train_idx, val_idx) in enumerate(kfold.split(np.arange(len(full_dataset)))):
        fold_num = fold + 1
        print(f"\n{'='*20} Fold {fold_num}/{k_folds} {'='*20}")
        
        # Create fold directory
        fold_dir = os.path.join(cv_dir, f"fold_{fold_num}")
        os.makedirs(fold_dir, exist_ok=True)
        
        # Create data loaders for this fold
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
        val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)
        
        train_loader = DataLoader(
            full_dataset, batch_size=int(best_params['batch_size']), 
            sampler=train_subsampler, num_workers=0
        )
        
        val_loader = DataLoader(
            val_dataset, batch_size=int(best_params['batch_size']), 
            sampler=val_subsampler, num_workers=0
        )
        
        # Create model
        model = create_model(num_classes, dropout_rate=best_params['dropout_rate'])
        model = set_trainable_layers(model, best_params['unfreeze_layers'])
        
        # Setup optimizer
        if best_params['optimizer'] == 'Adam':
            optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                                  lr=best_params['learning_rate'])
        else:
            optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=best_params['learning_rate'], momentum=0.9)
        
        # Setup loss and scheduler
        criterion = nn.CrossEntropyLoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                       factor=0.5, patience=5, verbose=True)
        
        # Train model for this fold
        model_path = os.path.join(fold_dir, f'model_fold_{fold_num}.pth')
        model, history = train_model(model, train_loader, val_loader, criterion, optimizer, 
                                    scheduler=scheduler, num_epochs=30, path=model_path)
        
        # Plot history
        plot_history(history, save_path=os.path.join(fold_dir, f"fold_{fold_num}"))
        
        # Evaluate model
        accuracy, report = evaluate_model(model, val_loader, class_names, 
                                         save_path=os.path.join(fold_dir, f"fold_{fold_num}"))
        
        # Store fold results
        fold_result = {
            'fold': fold_num,
            'accuracy': accuracy,
            'f1': report['weighted avg']['f1-score'],
            'per_class_f1': {cls: report[cls]['f1-score'] for cls in class_names}
        }
        
        fold_results.append(fold_result)
        all_metrics.append({
            'accuracy': accuracy,
            'f1': report['weighted avg']['f1-score']
        })
    
    # Calculate cross-validation metrics
    cv_df = pd.DataFrame(all_metrics)
    cv_summary = {
        'mean_accuracy': cv_df['accuracy'].mean(),
        'std_accuracy': cv_df['accuracy'].std(),
        'mean_f1': cv_df['f1'].mean(),
        'std_f1': cv_df['f1'].std(),
        'fold_results': fold_results
    }
    
    # Save CV summary
    with open(os.path.join(cv_dir, 'cv_summary.json'), 'w') as f:
        json.dump(cv_summary, f, indent=4)
    
    # Plot CV results
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    folds = [r['fold'] for r in fold_results]
    accuracies = [r['accuracy'] for r in fold_results]
    plt.bar(folds, accuracies)
    plt.axhline(y=cv_summary['mean_accuracy'], color='r', linestyle='-', 
               label=f"Mean: {cv_summary['mean_accuracy']:.4f} ± {cv_summary['std_accuracy']:.4f}")
    plt.xlabel('Fold')
    plt.ylabel('Accuracy')
    plt.title('Cross-Validation Accuracy by Fold')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    f1_scores = [r['f1'] for r in fold_results]
    plt.bar(folds, f1_scores)
    plt.axhline(y=cv_summary['mean_f1'], color='r', linestyle='-', 
               label=f"Mean: {cv_summary['mean_f1']:.4f} ± {cv_summary['std_f1']:.4f}")
    plt.xlabel('Fold')
    plt.ylabel('F1 Score')
    plt.title('Cross-Validation F1 Score by Fold')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(cv_dir, 'cv_results.png'))
    plt.close()
    
    print("\n" + "="*50)
    print("Cross-Validation Complete")
    print(f"Mean Accuracy: {cv_summary['mean_accuracy']:.4f} ± {cv_summary['std_accuracy']:.4f}")
    print(f"Mean F1 Score: {cv_summary['mean_f1']:.4f} ± {cv_summary['std_f1']:.4f}")
    
    return cv_summary

    #***************** Main Function *************************
def main():
    print("Starting Recycling Material Classification with ResNet-50")
    
    # 1. Run hyperparameter search
    print("\nRunning hyperparameter search...")
    best_params = run_hyperparameter_search(num_trials=15)
    
    # Uncomment this to use predefined best parameters instead of running the search
    # best_params = {
    #     'learning_rate': 1e-4,
    #     'batch_size': 32,
    #     'optimizer': 'Adam',
    #     'dropout_rate': 0.2,
    #     'unfreeze_layers': 'all'
    # }
    
    # 2. Train final model with best parameters
    print("\nTraining final model with best parameters...")
    final_model, test_accuracy, test_report = train_final_model(best_params)
    
    # # 3. Run cross-validation to verify results
    # print("\nRunning cross-validation to verify results...")
    # cv_summary = run_cross_validation(best_params, k_folds=5)
    
    # 4. Print final summary
    print("\n" + "="*60)
    print("EXPERIMENT SUMMARY")
    print("="*60)
    print(f"Model: ResNet-50")
    print(f"Best Model Test Accuracy: {test_accuracy:.4f}")
    print(f"Best Model Test F1 Score: {test_report['weighted avg']['f1-score']:.4f}")
    print(f"Cross-Validation Mean Accuracy: {cv_summary['mean_accuracy']:.4f} ± {cv_summary['std_accuracy']:.4f}")
    print(f"Cross-Validation Mean F1: {cv_summary['mean_f1']:.4f} ± {cv_summary['std_f1']:.4f}")
    print("="*60)
    print(f"Full results saved to {RESULTS_DIR}")
    
    return final_model, test_report, cv_summary

if __name__ == "__main__":
    main()

PyTorch version: 2.6.0
MPS available: True
Using MPS device
Starting Recycling Material Classification with ResNet-50

Running hyperparameter search...
Dataset loaded: 10500 training, 3000 validation, 1500 test images
Classes: ['aerosol_cans', 'aluminum_food_cans', 'aluminum_soda_cans', 'cardboard_boxes', 'cardboard_packaging', 'clothing', 'coffee_grounds', 'disposable_plastic_cutlery', 'eggshells', 'food_waste', 'glass_beverage_bottles', 'glass_cosmetic_containers', 'glass_food_jars', 'magazines', 'newspaper', 'office_paper', 'paper_cups', 'plastic_cup_lids', 'plastic_detergent_bottles', 'plastic_food_containers', 'plastic_shopping_bags', 'plastic_soda_bottles', 'plastic_straws', 'plastic_trash_bags', 'plastic_water_bottles', 'shoes', 'steel_food_cans', 'styrofoam_cups', 'styrofoam_food_containers', 'tea_bags']

Trial 1/15
Parameters: {'learning_rate': 0.0001, 'batch_size': 16, 'optimizer': 'Adam', 'dropout_rate': 0.5, 'unfreeze_layers': 'none'}
Dataset loaded: 10500 training, 3000 va

Training: 100%|███████████████████████████████| 657/657 [01:43<00:00,  6.32it/s]


Train Loss: 2.9354 Acc: 0.3462


Validation: 100%|█████████████████████████████| 188/188 [00:25<00:00,  7.48it/s]


Val Loss: 2.5156 Acc: 0.6063 F1: 0.6027
Validation loss decreased (inf --> 2.515558). Saving model...
Epoch 2/30


...

Train Loss: 0.1220 Acc: 0.9547


Validation: 100%|███████████████████████████████| 47/47 [00:25<00:00,  1.88it/s]


Val Loss: 0.3550 Acc: 0.8913 F1: 0.8907
EarlyStopping counter: 10 out of 10
Early stopping triggered
Training completed in 147m 16s
Training history plot saved to resnet50_results/cross_validation/fold_5/fold_5_history.png


Evaluating: 100%|███████████████████████████████| 47/47 [00:25<00:00,  1.87it/s]


Evaluation Accuracy: 0.8827

Classification Report:
                            precision    recall  f1-score   support

              aerosol_cans       0.95      0.95      0.95        87
        aluminum_food_cans       0.59      0.64      0.61       110
        aluminum_soda_cans       0.90      0.93      0.91        94
           cardboard_boxes       0.68      0.72      0.70       100
       cardboard_packaging       0.67      0.61      0.64        90
                  clothing       0.91      0.97      0.94        96
            coffee_grounds       0.95      1.00      0.98       101
disposable_plastic_cutlery       0.97      0.97      0.97        92
                 eggshells       0.99      0.98      0.99       105
                food_waste       0.96      0.98      0.97        89
    glass_beverage_bottles       0.84      0.91      0.88       100
 glass_cosmetic_containers       0.94      0.94      0.94        97
           glass_food_jars       0.94      0.93      0.93      